## 实验结果
model | acc（%） | total params |
---|---|---|
实验2：baseline(new method - attention)|85.47|6,000,301
实验3：baseline(LSTM)|86.09|6,219,777
实验4：baseline(LSTM+Attention)|89.08|6,232,777
实验1：__new method__(embeddings+Attention)|__89.12__|6,020,251
1. 实验1是提出的新模型，结果最优，参数少；<br/>实验2是去掉新模型中的注意力机制后的模型，baseline；<br/>实验3是LSTM模型,实验4是LSTM+attention模型，准确率和新模型不相上下，但是比新模型参数多200,000个左右。<br/>ps:实验3结果不稳定：出现过84%，87.22%.


## 实验
#### 0 数据预处理

In [2]:
# 加载数据
import numpy as np
from keras.models import Model
from keras import backend as K
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers.core import Lambda,RepeatVector,Lambda,Reshape
from keras.layers import Dense,Dropout,Embedding,LSTM,Bidirectional,GlobalAveragePooling1D,Input,GlobalMaxPooling1D,Multiply
from keras.datasets import imdb
from keras.callbacks import EarlyStopping
import keras
from numpy.random import seed
seed(20190419)    
from tensorflow import set_random_seed 
set_random_seed(20190419)

max_features= 20000
maxlen=250
batch_size=32
embedding_dims=300
epochs=20

(x_train,y_train),(x_test,y_test)=imdb.load_data("F:/Anaconda3/word vector and sentiment analysis/data/imdb.npz",num_words=max_features)

x_train=sequence.pad_sequences(x_train,maxlen=maxlen)
x_test=sequence.pad_sequences(x_test,maxlen=maxlen)

y_train=np.array(y_train)
y_test=np.array(y_test)

Using TensorFlow backend.


#### 1 新方法 embeddings+attention
模型结构：&nbsp;input&nbsp; ->&nbsp; embeddings &emsp;&emsp;&emsp;&emsp;   -> &nbsp; embeddings&nbsp; * &nbsp; attention emb &nbsp;->&nbsp; sum &nbsp;->&nbsp; predict  &nbsp;  <br/>
&emsp; &emsp;  &emsp;&emsp;&emsp;&emsp;&emsp; -> attention emb -> 归一化                     

In [3]:
input_words = Input(shape=(maxlen,), dtype='int32')
input_vecs = Embedding(max_features, embedding_dims)(input_words)
input_atten = Embedding(max_features,1,name='atten_L1')(input_words)
input_atten_norm=Lambda(lambda x: K.exp(input_atten)/K.sum(K.exp(input_atten)))(input_atten)  # attention 归一化
input_atten_1=Lambda(lambda x: K.repeat_elements(input_atten_norm, embedding_dims, 2))(input_atten_norm)
temp_layers= Multiply()([input_vecs,input_atten_1])   # 单词 对应的 word vectors 和 attention 相乘
sumLayer=Lambda(lambda x: K.sum(x, axis=2))(temp_layers)  # axis=1时，是单词词向量求和；此模型选择axis=2,是对每个单词词向量各项的相加，每个单词对应一个值，
predict=Dense(1, activation='sigmoid')(sumLayer)
model = Model(inputs=input_words, outputs=predict)
print(model.summary())
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
callbacks = EarlyStopping(monitor='val_loss', patience=2, verbose=0)
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test),
         callbacks=[callbacks])

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 250)          0                                            
__________________________________________________________________________________________________
atten_L1 (Embedding)            (None, 250, 1)       20000       input_1[0][0]                    
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 250, 1)       0           atten_L1[0][0]                   
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 250, 300)     6000000     input_1[0][0]                    
__________________________________________________________________________________________________
lambda_2 (

#### 2 baseline: embeddings （ "no" attention）
模型结构：input -> embeddings -> sum -> predict

In [4]:
input_words = Input(shape=(maxlen,), dtype='int32')
input_vecs = Embedding(max_features, embedding_dims)(input_words)
sumLayer=Lambda(lambda x: K.sum(x, axis=1))(input_vecs)
predict=Dense(1, activation='sigmoid')(sumLayer)
model = Model(inputs=input_words, outputs=predict)
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
callbacks = EarlyStopping(monitor='val_loss', patience=2, verbose=0)
print(model.summary())
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test),
         callbacks=[callbacks])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 250)               0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 250, 300)          6000000   
_________________________________________________________________
lambda_4 (Lambda)            (None, 300)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 301       
Total params: 6,000,301
Trainable params: 6,000,301
Non-trainable params: 0
_________________________________________________________________
None
Train on 25000 samples, validate on 25000 samples
Epoch 1/20
25000/25000 [==============================] - 187s 7ms/step - loss: 0.3898 - acc: 0.8473 - val_loss: 0.3688 - val_acc: 0.8492
Epoch 2/20
25000/25000 [==============================] - 189s 8ms/step

#### 3 baseline: embeddings + LSTM
模型结构：input -> embeddings -> LSTM -> predict

In [6]:
input_words = Input(shape=(maxlen,), dtype='int32')
input_vecs = Embedding(max_features, embedding_dims)(input_words)
lstm=LSTM(128, dropout=0.2, recurrent_dropout=0.2)(input_vecs)
predict=Dense(1, activation='sigmoid')(lstm)
model = Model(inputs=input_words, outputs=predict)
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())
callbacks = EarlyStopping(monitor='val_loss', patience=2, verbose=0)
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test),
         callbacks=[callbacks])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 250)               0         
_________________________________________________________________
embedding_4 (Embedding)      (None, 250, 300)          6000000   
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               219648    
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 129       
Total params: 6,219,777
Trainable params: 6,219,777
Non-trainable params: 0
_________________________________________________________________
None
Train on 25000 samples, validate on 25000 samples
Epoch 1/20
25000/25000 [==============================] - 855s 34ms/step - loss: 0.5465 - acc: 0.7217 - val_loss: 0.4330 - val_acc: 0.8010
Epoch 2/20
25000/25000 [==============================] - 858s 34ms/st

#### 4 baseline: embeddings + LSTM
模型结构：input -> embeddings -> LSTM -> attention -> predict

In [7]:
from keras.engine.topology import Layer
from keras import initializers


class AttLayer(Layer):
    def __init__(self, attention_dim):
        self.init = initializers.get('normal')
        self.supports_masking = True
        self.attention_dim = attention_dim
        super(AttLayer, self).__init__()

    def build(self, input_shape):
        assert len(input_shape) == 3
        self.W = K.variable(self.init((input_shape[-1], self.attention_dim)))
        self.b = K.variable(self.init((self.attention_dim, )))
        self.u = K.variable(self.init((self.attention_dim, 1)))
        self.trainable_weights = [self.W, self.b, self.u]
        super(AttLayer, self).build(input_shape)

    def compute_mask(self, inputs, mask=None):
        return mask

    def call(self, x, mask=None):
        # size of x :[batch_size, sel_len, attention_dim]
        # size of u :[batch_size, attention_dim]
        # uit = tanh(xW+b)
        uit = K.tanh(K.bias_add(K.dot(x, self.W), self.b))
        ait = K.dot(uit, self.u)
        ait = K.squeeze(ait, -1)

        ait = K.exp(ait)

        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            ait *= K.cast(mask, K.floatx())
        ait /= K.cast(K.sum(ait, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        ait = K.expand_dims(ait)
        weighted_input = x * ait
        output = K.sum(weighted_input, axis=1)

        return output

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1])

    
# maxlen=250
input_words = Input(shape=(maxlen,), dtype='int32')
input_vecs = Embedding(max_features, embedding_dims)(input_words)
lstm=LSTM(128, dropout=0.2, recurrent_dropout=0.2,return_sequences = True)(input_vecs)
attention = AttLayer(100)(lstm)
predict=Dense(1, activation='sigmoid')(attention)
model = Model(inputs=input_words, outputs=predict)
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())
callbacks = EarlyStopping(monitor='val_loss', patience=2, verbose=0)
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test),
         callbacks=[callbacks])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 250)               0         
_________________________________________________________________
embedding_5 (Embedding)      (None, 250, 300)          6000000   
_________________________________________________________________
lstm_3 (LSTM)                (None, 250, 128)          219648    
_________________________________________________________________
att_layer_1 (AttLayer)       (None, 128)               13000     
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 129       
Total params: 6,232,777
Trainable params: 6,232,777
Non-trainable params: 0
_________________________________________________________________
None
Train on 25000 samples, validate on 25000 samples
Epoch 1/20
25000/25000 [==============================] - 886s 35ms/step - lo